# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e  \
5   721.951953  0.366076  1234  722.318029  721.585877  1444.636058   
6   285.563253  0.772903  1234  286.336156  284.790350   572.672311   
7   820.251860 -0.639528  1234  819.612332  820.891388  1639.224664   
8   789.289091 -0.705774  1234  788.583317  789.994864  1577.166634   
9   937.937898  0.030863  1234  937.968761  937.907036  1875.937523   
10  280.330547 -0.762713  1234  279.567834  281.093260   559.135667   
11  310.631917  0.396587  1234  311.028503  310.235330   622.057007   
12  551.469244  0.698657  1234  552.167901  550.770587  1104.335801   
13  496.136239 -0.239626  1234  495.896612  496.375865   991.793224   
14  842.982500 -0.633479  1234  842.349021  843.615979  1684.698042   

              f  
5   1443.171753  
6    569.580699  
7   1641.782777  
8   1579.989729  
9   1875.814071  
10   562.186520  
11   620.470660  
12  1101.541173  
13   992.751730  
14  1687.231959

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -722.318029   721.585877
6   -286.336156   284.790350
7   -819.612332   820.891388
8   -788.583317   789.994864
9   -937.968761   937.907036
10  -279.567834   281.093260
11  -311.028503   310.235330
12  -552.167901   550.770587
13  -495.896612   496.375865
14  -842.349021   843.615979

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -722.318029   721.585877
6   -286.336156   284.790350
7   -819.612332   820.891388
8   -788.583317   789.994864
9   -937.968761   937.907036
10  -279.567834   281.093260
11  -311.028503   310.235330
12  -552.167901   550.770587
13  -495.896612   496.375865
14  -842.349021   843.615979

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-722.31802908,  721.58587656],
       [-286.33615564,  284.79034962],
       [-819.61233224,  820.89138833],
       [-788.58331714,  789.99486439],
       [-937.96876125,  937.90703551],
       [-279.56783361,  281.09326004],
       [-311.02850336,  310.23532997],
       [-552.16790067,  550.77058661],
       [-495.89661217,  496.37586498],
       [-842.34902076,  843.61597932]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5    1442.636058  -1443.171753
6     570.672311   -569.580699
7    1637.224664  -1641.782777
8    1575.166634  -1579.989729
9    1873.937523  -1875.814071
10    557.135667   -562.186520
11    620.057007   -620.470660
12   1102.335801  -1101.541173
13    989.793224   -992.751730
14   1682.698042  -1687.231959

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11       False        True     False
12       False        True     False
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf